## Extract FL Data

In [156]:
import numpy as np
import pandas as pd

In [157]:
train_df = pd.read_csv('../../data_analysis/fd001/fd001-raw_train.csv', sep=' ')
test_df = pd.read_csv('../../data_analysis/fd001/fd001-raw_test.csv', sep=' ')
train_labels_df = pd.read_csv('../../data_analysis/fd001/fd001-training_labels.csv', sep=' ')

In [158]:
test_df = test_df.groupby('ID').last().reset_index()
test_labels_df = pd.read_csv('../../TED/CMAPSSData/RUL_FD001.txt', sep=' ', header = None)

In [159]:
columns = train_df.columns
ms_used = ['RUL']
for i in range(1, 22):
    if i not in [1, 5, 6, 9, 10, 14, 16, 18, 19]:
        ms_used.append('SensorMeasure' + str(i))

In [160]:
test_labels_df.drop(columns=[1], inplace = True)
test_labels_df.columns = ['RUL']

In [161]:
train_labels_df[columns] = train_df[columns]
test_labels_df[columns] = test_df[columns]

In [162]:
train_df = train_labels_df.copy()
test_df = test_labels_df.copy()

In [163]:
train_gb = train_df.groupby(['ID'], sort = False)

In [164]:
# Args: List of pcts -> ex: [0.75, 0.2, 0.05]
def split_data(pcts, mode = 'train'):

    idx_remaining = [i for i in range(1, len(train_gb) + 1)]
    nums = []
    indices_concat = []
    for pct in pcts:
        nums.append(round(pct * len(train_gb)))
    n_workers = len(pcts)

    for i in range(n_workers):
        idx_worker = np.sort(np.random.choice(idx_remaining, nums[i], replace = False))
        indices_concat.append(idx_worker) 
        idx_remaining = np.setdiff1d(idx_remaining, idx_worker)

    for i in range(len(indices_concat)):
        id_list = indices_concat[i]
        df = train_gb.get_group(id_list[0])
        for j in range(1, len(id_list)):
            df = pd.concat([df, train_gb.get_group(id_list[j])])
        df[ms_used].to_csv('./data/train_partition_' + str(i) + '.csv', sep=' ', index_label='id')

split_data([0.25, 0.25, 0.25, 0.25], 'train')

### Test data - convert format

In [165]:
test_df

,RUL,ID,Cycle,SensorMeasure2,SensorMeasure3,SensorMeasure4,SensorMeasure7,SensorMeasure8,SensorMeasure11,SensorMeasure12,SensorMeasure13,SensorMeasure15,SensorMeasure17,SensorMeasure20,SensorMeasure21
0,112,1,31,642.58,1581.22,1398.91,554.42,2388.08,47.23,521.79,2388.06,8.4024,393,38.81,23.3552
1,98,2,49,642.55,1586.59,1410.83,553.52,2388.10,47.67,521.74,2388.09,8.4505,391,38.81,23.2618
2,69,3,126,642.88,1589.75,1418.89,552.59,2388.16,47.88,520.83,2388.14,8.4119,395,38.93,23.2740
3,82,4,106,642.78,1594.53,1406.88,552.64,2388.13,47.65,521.88,2388.11,8.4634,395,38.58,23.2581
4,91,5,98,642.27,1589.94,1419.36,553.29,2388.10,47.46,521.00,2388.15,8.4362,394,38.75,23.4117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,137,96,97,642.30,1590.88,1397.94,553.99,2388.03,47.14,522.30,2388.01,8.4110,391,38.96,23.4606
96,82,97,134,642.59,1582.96,1410.92,554.05,2388.06,47.38,521.58,2388.06,8.4500,395,38.61,23.2953
97,59,98,121,642.68,1599.51,1415.47,553.44,2388.13,47.66,521.53,2388.09,8.4235,394,38.76,23.3608
98,117,99,97,642.00,1585.03,1397.98,554.75,2388.01,47.26,521.82,2388.02,8.4003,391,38.95,23.3595


In [166]:
test_df[ms_used].to_csv('./data/test_partition.csv', sep=' ', index_label='id')